In [ ]:
from IPython.utils import capture
from IPython.display import clear_output
import os

#@markdown ## 1. ComfyUI 설치 
#@markdown ##### Install ComfyUI
#@markdown <details>
#@markdown   <summary>설치된 커스텀 노드 자세히 펼쳐보기 (details)</summary>
#@markdown
#@markdown **WD14-Tagger** 이미지->프롬프트
#@markdown > https://github.com/pythongosssss/ComfyUI-WD14-Tagger 
#@markdown
#@markdown **MTB Nodes** 이미지 / 애니메이션관련
#@markdown > https://github.com/melMass/comfy_mtb
#@markdown
#@markdown **rgthree** 편의성
#@markdown > https://github.com/rgthree/rgthree-comfy
#@markdown
#@markdown **was-node-suite** 편의성
#@markdown > https://github.com/WASasquatch/was-node-suite-comfyui
#@markdown
#@markdown **ComfyUI_ADV_CLIP_emb** A1111 프롬프트 가중치
#@markdown > https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb
#@markdown
#@markdown **ComfyUI-Easy-Use** Dynamic Prompt (Wildcards)
#@markdown > https://github.com/yolain/ComfyUI-Easy-Use
#@markdown
#@markdown **comfyui-lama-remover** Lama (remove object)
#@markdown > https://github.com/Layer-norm/comfyui-lama-remover
#@markdown
#@markdown **ComfyUI_essentials** Image, Mask
#@markdown > https://github.com/cubiq/ComfyUI_essentials
#@markdown
#@markdown </details>

HomePath = '/content'
Workspace = 'install_comfyui'

print('install...')
if not os.path.isdir(f'{HomePath}/{Workspace}'):
  with capture.capture_output() as cap:
    !wget https://huggingface.co/ninjaneural/webui/resolve/main/direct/install_comfyui_pack_20240516.tar -O install_comfyui.tar
    !tar -C {HomePath} -xvf install_comfyui.tar
    !rm install_comfyui.tar
    !wget https://raw.githubusercontent.com/ninjaneural/comfyui/master/misc/extra_model_paths.yaml -O {HomePath}/{Workspace}/extra_model_paths.yaml
    !sed -i -e "/    init_custom_nodes()/a\    folder_paths.add_model_folder_path('AnimateDiffEvolved_Models', '/content/animatediff/models')\n    folder_paths.add_model_folder_path('AnimateDiffMotion_LoRA', '/content/animatediff/motion_lora')" {HomePath}/{Workspace}/main.py
    %cd {HomePath}/{Workspace}

    !git reset --hard
    !git pull
    %cd ./custom_nodes/ComfyUI-Easy-Use
    !git pull
    %cd {HomePath}/{Workspace}

print("설치완료. 실행환경 설치을 진행해주세요")

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 2. 실행환경 설치
#@markdown ##### Install Env

%cd /content

!mkdir /content/models
!mkdir /content/models/checkpoints
!mkdir /content/models/controlnet_models
!mkdir /content/models/upscale_models
!mkdir /content/models/loras
!mkdir /content/models/vae
!mkdir /content/models/clip_vision
!mkdir /content/models/clip

print('install library...')
with capture.capture_output() as cap:
  !pip install kornia
  !pip install torchsde
  !pip install insightface
  !pip install timm

  !pip install diffusers==0.25.0
  !pip install accelerate
  !pip install gradio==4.24.0
  !pip install av
  !pip install einops==0.7.0
  !pip install onnxruntime
  !pip install omegaconf>=2.0.0
  !pip install fvcore
  !pip install ultralytics
  !pip install rembg

  !pip install segment_anything
  !pip install addict
  !pip install yapf
  
  !apt install aria2
  
  !pip install spandrel

print("설치완료. 모델 다운로드를 진행해주세요")


In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 3. 모델 다운로드
#@markdown ##### Model Download

#@markdown **모델(checkpoint) URL (colab에 다운받는 모델)**
Checkpoint_Url = 'https://huggingface.co/lllyasviel/flux1-dev-bnb-nf4/resolve/main/flux1-dev-bnb-nf4-v2.safetensors?download=true' #@param {type:"string"}
#@markdown **모델 파일명**
Checkpoint_Filename = 'flux1-dev-bnb-nf4-v2.safetensors' #@param {type:"string"}
#@markdown **모델 다운로드**
Checkpoint_Download = True #@param {type:"boolean"}

#@markdown -----
#@markdown *컨트롤넷 모델*

#@markdown **FluxControlNet 기본 모델 다운로드**
FluxControlNet = False #@param {type:"boolean"}

#@markdown **SVD 모델 다운로드**
SVD_Checkpoint = False #@param {type:"boolean"}

#@markdown **insightface 설치 (reactor 노드사용)**
Insightface = False #@param {type:"boolean"}

print('flux controlnet download...')
with capture.capture_output() as cap:

  if FluxControlNet:
    !mkdir {HomePath}/{Workspace}/xlabs
    !mkdir {HomePath}/{Workspace}/xlabs/controlnets
    !mkdir {HomePath}/{Workspace}/xlabs/loras
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/XLabs-AI/flux-controlnet-collections/resolve/main/flux-canny-controlnet_v2.safetensors -d /content/models/xlabs/controlnets -o flux-canny-controlnet_v2.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/XLabs-AI/flux-controlnet-collections/resolve/main/flux-depth-controlnet_v2.safetensors -d /content/models/xlabs/controlnets -o flux-depth-controlnet_v2.safetensors

# svd checkpoint
print('svd download...')
with capture.capture_output() as cap:
  if SVD_Checkpoint:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid/resolve/main/svd.safetensors -d /content/models/checkpoints -o svd.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors -d /content/models/checkpoints -o svd_xt.safetensors

# checkpoint
print('checkpoint download...')
with capture.capture_output() as cap:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d /content/models/checkpoints -o {Checkpoint_Filename}

  # upscaler
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/models/upscale_models -o 4x-UltraSharp.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gemasai/4x_NMKD-Siax_200k/resolve/main/4x_NMKD-Siax_200k.pth -d /content/models/upscale_models -o 4x_NMKD-Siax_200k.pth

  # vae
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/ae.safetensors -d /content/models/vae -o ae.safetensors

  # lora
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/XLabs-AI/flux-RealismLora/resolve/main/lora.safetensors -d /content/models/loras -o flux-RealismLora.safetensors


%cd {HomePath}/{Workspace}

print("다운로드완료. ComfyUI 실행을 진행해주세요")

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## (옵션) 추가 커스텀 노드 설치
#@markdown ##### (Optional) Install Custom Nodes
#@markdown * ReActor
ReActor = True #@param {type:"boolean"}

#@markdown * DreamTalk
DREAMTALK = False #@param {type:"boolean"}

#@markdown * IDM-VTON
IDM_VTON = False #@param {type:"boolean"}

magicword = "onymous"

#@markdown <br/>
#@markdown <details>
#@markdown   <summary>추가 커스텀노드 자세히 펼쳐보기 (details)</summary>
#@markdown
#@markdown **comfyui_segment_anything** Segment Anything, DINO
#@markdown > https://github.com/storyicon/comfyui_segment_anything
#@markdown
#@markdown **ComfyUI-Video-Matting** 배경 분리 Robust Video Matting (RVM)
#@markdown > https://github.com/Fannovel16/ComfyUI-Video-Matting
#@markdown
#@markdown </details>

%cd {HomePath}/{Workspace}/custom_nodes


print('custom nodes download...')
with capture.capture_output() as cap:
  !git clone https://github.com/storyicon/comfyui_segment_anything
  !git clone https://github.com/Fannovel16/ComfyUI-Video-Matting
  !git clone https://github.com/XLabs-AI/x-flux-comfyui
  !git clone https://github.com/comfyan{magicword}/ComfyUI_bitsandbytes_NF4
  %cd ComfyUI_bitsandbytes_NF4
  !pip install -r requirements.txt
  %cd ..

  if DREAMTALK:
    !git clone https://github.com/neuralninja22/ComfyUI_DDreamtalk
    %cd ComfyUI_DDreamtalk
    !pip install -r requirements.txt
    %cd ..
  
  if IDM_VTON:
    !git clone https://github.com/neuralninja22/IDM-VTON
    !apt install aria2

    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/openpose/ckpts/body_pose_model.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/levihsu/OOTDiffusion/resolve/main/checkpoints/openpose/ckpts/body_pose_model.pth -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/openpose/ckpts -o body_pose_model.pth
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing/parsing_atr.onnx
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/humanparsing/parsing_atr.onnx -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing -o parsing_atr.onnx
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing/parsing_lip.onnx
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/humanparsing/parsing_lip.onnx -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing -o parsing_lip.onnx
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/densepose/model_final_162be9.pkl
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/densepose/model_final_162be9.pkl -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/densepose -o model_final_162be9.pkl
    
  if not ReActor:
    !mv comfyui-reactor-node ../../

%cd {HomePath}/{Workspace}

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## (옵션) SDXL 추가 커스텀 노드 설치
#@markdown ##### (Optional) Install SDXL Custom Nodes

#@markdown **comfyui-inpaint-nodes** Fooocus inpaint
#@markdown > https://github.com/Acly/comfyui-inpaint-nodes

%cd {HomePath}/{Workspace}/custom_nodes

print('custom nodes download...')
with capture.capture_output() as cap:
  !git clone https://github.com/Acly/comfyui-inpaint-nodes
  
%cd {HomePath}/{Workspace}

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 4. ComfyUI 실행
#@markdown ##### Run ComfyUI

#@markdown -----
#@markdown *터널링*

Tunneling = "LocalTunnel" #@param ["LocalTunnel"]

#@markdown **임시서버 사용** (LocalTunnel 서버가 동작하지 않을때 이용해주세요)
TunnelTempServer = False #@param {type:"boolean"}

%cd {HomePath}/{Workspace}

if Tunneling == "LocalTunnel":
  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()

    command = ["lt", "--port", "{}".format(port)]
    if TunnelTempServer:
      command = ["lt", "--host", "http://lt.dabayo.com", "--port", "{}".format(port)]
    else:
      print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    def run_process(count_down):
      p = subprocess.Popen(command, stdout=subprocess.PIPE)
      for line in p.stdout:
        url_output = line.decode()
        your_domain = url_output[13:]
        try:
          response = urllib.request.urlopen(your_domain, timeout=15)
          print("your url : " + your_domain)
          data = response.read().decode('utf8').strip("\n")
        except Exception as e:
          print(f"General Error: {e}")
          print("> retry")
          p.kill()
          time.sleep(3)
          if count_down > 0:
            run_process(count_down-1)
          else:
            print("> localtunnel failed")


    run_process(10)

  threading.Thread(target=iframe_thread, daemon=True, args=(18188,)).start()

!python main.py --port 18188 --dont-print-server